### 查看每列缺失值的数量和比率函数

In [1]:
def missing(combine):
    mark=0
    for data in combine:
        dic={}
        dic['Alength']=len(data)
        for i in data.isnull().any()[data.isnull().any()].index:
            dic[i]=[str(len(data[data[i].isnull()]))+'   '+str(round(100*len(data[data[i].isnull()])/len(data),3))+'%']
        if mark==100:
            nantest=pd.DataFrame(dic)
            break
        else:
            nantrain=pd.DataFrame(dic)
        mark=100
    return nantrain,nantest

## 删除instance_id重复的列，组合train和test到一起进行预处理，一天下午4点到第二天下午4点为新的一天

In [6]:
import pandas as pd
import numpy as np
#['predict_category_property', 'item_category_list', 'item_property_list']
train=pd.read_csv(r'D:\Data\IJCAI\train.csv',na_values=-1)
test=pd.read_csv(r'D:\Data\IJCAI\test.csv',na_values=-1)
train.drop_duplicates(inplace=True)
train=train.sort_values(by=['instance_id'])
train.index=np.arange(len(train))
for ind in train.index:                     #删除完全重复的记录并对只有instance_id重复的记录重新赋值
    if ind<(len(train)-1):
        if train.loc[ind,'instance_id']==train.loc[(ind+1),'instance_id']:
            train.loc[(ind+1),'instance_id']=train.loc[ind,'instance_id']+1
print('predict_category_property列缺失的行数：',len(train[train['predict_category_property'].isnull()]))
train.drop(list(train[train['predict_category_property'].isnull()].index),axis=0,inplace=True)

train['timestamp']=pd.to_datetime(train['context_timestamp'].copy(),unit='s')
train['day']=train['timestamp'].dt.day
test['timestamp']=pd.to_datetime(test['context_timestamp'].copy(),unit='s')
test['day']=test['timestamp'].dt.day
 
is_trade=train['is_trade']                                  #将train和test组合到一起进行处理
train.drop(['is_trade'],axis=1,inplace=True)
train['is_trade']=is_trade
test['is_trade']=1000
train=pd.concat([train,test],axis=0)
train.index=np.arange(len(train))

count=17
train['newday']=0
for day in [17,18,19,20,21,22,23,24]:                                        #一天下午4点到第二天下午4点
    lis=(train[(train['timestamp']>='2018-09-%s 16:00:00'%(day))&(train['timestamp']<'2018-09-%s 16:00:00'%(day+1))].index)
    train.loc[lis,'newday']=count
    count+=1

predict_category_property列缺失的行数： 4


## “前一天”商品的转换率和“前两天”的转换率

In [9]:
train['num_conversion']=0
train['num_conversiontd']=0
for newday in set(train['newday'].values):
    if newday<18:
        continue
    else:
        for u in set(train[train['newday']==newday]['item_id'].values):
            if u in train[train['newday']==(newday-1)]['item_id'].values:
                train.loc[train[(train['item_id']==u)&(train['newday']==newday)].index,'num_conversion']=\
                len(train[(train['item_id']==u)&(train['newday']==(newday-1))&(train['is_trade']==1)])/len(train[(train['item_id']==u)&(train['newday']==(newday-1))])
            if newday<19:
                continue
            else:
                if u in train[(train['newday']==(newday-1))|(train['newday']==(newday-2))]['item_id'].values:
                    train.loc[train[(train['item_id']==u)&(train['newday']==newday)].index,'num_conversiontd']=\
                    (len(train[(train['item_id']==u)&(train['newday']==(newday-1))&(train['is_trade']==1)])+\
                     len(train[(train['item_id']==u)&(train['newday']==(newday-2))&(train['is_trade']==1)]))/(len(train[(train['item_id']==u)&(train['newday']==(newday-1))])+\
                                                                                                    len(train[(train['item_id']==u)&(train['newday']==(newday-2))]))
print('train has done！')
train.to_csv(r'D:\Data\IJCAI\no_fixedday\train_conversion.csv',index=False)

train is done！


## 提取"每天"用户点击数，商品被点击数，各个时间间隔,并将train和test重新分开，test按照最开始的顺序排序

In [10]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\train_conversion.csv',na_values=-1)
train=train.sort_values(by=['newday','user_id'])                               #同一天同一个用户的点击广告的次数——--用户的搜索次数和点击次数
h=train.groupby(by=['newday','user_id']).count()
lis=[]
for i,j in h.index:
    lis.extend([h.loc[i].loc[j]['instance_id']]*h.loc[i].loc[j]['instance_id'])
train['num_click']=lis

train=train.sort_values(by=['newday','item_id'])                              #同一天同一件商品被搜索点击次数-----商品的被搜索次数和受欢迎程度
h=train.groupby(by=['newday','item_id']).count()
lis=[]
for i,j in h.index:
    lis.extend([h.loc[i].loc[j]['instance_id']]*h.loc[i].loc[j]['instance_id'])
train['num_show']=lis
    
train=train.sort_values(by=['item_id','context_timestamp'])                 #同一件商品被搜索点击的时间间隔
train.index=np.arange(len(train))
lis=[]
for val in set(train['item_id'].values):
    lis.append(train[train['item_id']==val].index[-1])
temp=train['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
train['item_time_diff']=temp

train=train.sort_values(by=['newday','item_id','context_timestamp'])          #同一天同一件商品的被搜索点击的时间间隔
train.index=np.arange(len(train))
lis=[]
for d in [17,18,19,20,21,22,23,24]:
    for val in set(train[train['newday']==d]['item_id'].values):
        lis.append(train[(train['item_id']==val)&(train['newday']==d)].index[-1])
temp=train['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
train['newday_item_diff']=temp
    
train=train.sort_values(by=['newday','user_id','context_timestamp'])          #同一天相同用户的不同商品点击时间间隔
train.index=np.arange(len(train))
lis=[]
for d in [17,18,19,20,21,22,23,24]:
    for val in set(train[train['newday']==d]['user_id'].values):
        lis.append(train[(train['user_id']==val)&(train['newday']==d)].index[-1])
temp=train['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
train['newday_user_diff']=temp

train.index=np.arange(len(train))
train['num_item_newday']=0
for val in set(train['item_id'].values):               #商品被推荐并被点击的天数
    train.loc[list(train[train['item_id']==val].index),'num_item_newday']=len(set(train[train['item_id']==val]['newday'].values))
    
train['num_user_newday']=0
for val in set(train['user_id'].values):               #用户点击广告的天数
    train.loc[list(train[train['user_id']==val].index),'num_user_newday']=len(set(train[train['user_id']==val]['newday'].values))
##########################################################################################################################################
test=train[train['is_trade']==1000]
train.drop(list(train[train['is_trade']==1000].index),axis=0,inplace=True)
train.to_csv(r'D:\Data\IJCAI\no_fixedday\train_pre1.csv',index=False)
print('Train has being Done!')

test.drop(['is_trade'],axis=1,inplace=True)                  #test按照最初的顺序排序好提交结果
test.index=np.arange(len(test))
print('length of test is：',len(test))
new=pd.read_csv(r'D:\Data\IJCAI\test.csv')
print('length of new is：',len(new))
# print('Is set(new[instance_id].values)==set(test[instance_id].values)',set(new['instance_id'].values)==set(test['instance_id'].values))
lis=[]
for i in new.index:
    lis.extend(list(test[test['instance_id']==new.loc[i,'instance_id']].index))
test=test.loc[lis]
test.index=np.arange(len(test))
test.to_csv(r'D:\Data\IJCAI\no_fixedday\test_pre1.csv',index=False)

Train has being Done!


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


length of test is： 18371
length of new is： 18371


# 对缺失值进行填充

In [3]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\train_pre1.csv',na_values=-1)
test=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\test_pre1.csv',na_values=-1)
combine=[train,test]
nantrain,nantest=missing(combine)
nantrain

,Alength,item_brand_id,item_city_id,item_sales_level,shop_review_positive_rate,shop_score_delivery,shop_score_description,shop_score_service,user_age_level,user_gender_id,user_occupation_id,user_star_level
0,478107,473 0.099%,277 0.058%,913 0.191%,7 0.001%,59 0.012%,59 0.012%,59 0.012%,964 0.202%,12902 2.699%,964 0.202%,964 0.202%


In [4]:
nantest

,Alength,item_brand_id,item_city_id,item_sales_level,shop_score_delivery,shop_score_description,shop_score_service,user_age_level,user_gender_id,user_occupation_id,user_star_level
0,18371,18 0.098%,6 0.033%,35 0.191%,1 0.005%,1 0.005%,1 0.005%,18 0.098%,469 2.553%,18 0.098%,18 0.098%


In [4]:
# corr=train[['instance_id', 'item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', \
#             'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id','user_star_level', 'context_id', \
#             'context_timestamp', 'context_page_id','shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', \
#             'shop_score_service', 'shop_score_delivery', 'shop_score_description', 'timestamp', 'day', 'is_trade', 'num_conversion', \
#             'num_conversiontd', 'num_click', 'num_show', 'item_time_diff', 'day_item_diff', 'day_user_diff']].corr()
# corr[corr['shop_review_positive_rate']>0.1]['shop_review_positive_rate'].sort_values(ascending=False).head(6)#shop_score_description
train.index=np.arange(len(train))
test.index=np.arange(len(test))
for ind in train[train['shop_review_positive_rate'].isnull()].index:
    item_price_level=train.loc[ind,'item_price_level']
    train.loc[ind,'shop_review_positive_rate']=train[train['item_price_level']==item_price_level]['shop_review_positive_rate'].median()
# corr[corr['shop_score_delivery']>0.1]['shop_score_delivery'].sort_values(ascending=False).head(6)
#['shop_score_delivery','shop_score_description','shop_score_service']这四列是同时缺失的需要一起填充
#train[train['user_age_level'].isnull()][['is_trade','user_age_level','user_occupation_id','user_star_level']]
h=['shop_score_delivery','shop_score_description','shop_score_service']
combine=[train,test]
for data in combine:
    for hj in h:
        for ind in data[data[hj].isnull()].index:
            shop_review_positive_rate=data.loc[ind,'shop_review_positive_rate']
            item_price_level=data.loc[ind,'item_price_level']
            temp=data[(data['shop_review_positive_rate']==shop_review_positive_rate)&(data['item_price_level']==item_price_level)]
            if temp['shop_score_service'].isnull().any():
                data.loc[ind,'shop_score_service']=data[data['item_price_level']==item_price_level]['shop_score_service'].median()
            else:
                data.loc[ind,'shop_score_service']=temp['shop_score_service'].median()
            if temp['shop_score_delivery'].isnull().any():
                data.loc[ind,'shop_score_delivery']=data[data['item_price_level']==item_price_level]['shop_score_delivery'].median()
            else:
                data.loc[ind,'shop_score_delivery']=round(temp[~temp['shop_score_delivery'].isnull()]['shop_score_delivery'].median(),6)
            if temp['shop_score_description'].isnull().any():
                data.loc[ind,'shop_score_description']=data[data['item_price_level']==item_price_level]['shop_score_description'].median()
            else:
                data.loc[ind,'shop_score_description']=temp[~temp['shop_score_description'].isnull()]['shop_score_description'].median()
# corr[corr['user_age_level']>0.1]['user_age_level'].sort_values(ascending=False).head(6)
#['is_trade','user_age_level','user_occupation_id','user_star_level']这三列是同时缺失的需要一起填充
combine=[train,test]
h=['user_age_level','user_occupation_id','user_star_level']
for data in combine:
    for hj in h:
        for ind in data[data[hj].isnull()].index:
            shop_score_description=data.loc[ind,'shop_score_description']
            item_price_level=data.loc[ind,'item_price_level']
            temp=data[(data['shop_score_description']==shop_score_description)&(data['item_price_level']==item_price_level)]
            if temp['user_age_level'].isnull().any():
                data.loc[ind,'user_age_level']=data[data['item_price_level']==item_price_level]['user_age_level'].median()
            else:
                data.loc[ind,'user_age_level']=temp['user_age_level'].median()
            if temp['user_occupation_id'].isnull().any():
                if (data[data['shop_score_description']==shop_score_description]['user_occupation_id'].isnull().any())\
                and (data[data['item_price_level']==item_price_level]['user_occupation_id'].isnull().any()):
                    data.loc[ind,'user_occupation_id']=data.loc[ind+6,'user_occupation_id']
                elif (data[data['shop_score_description']==shop_score_description]['user_occupation_id'].isnull().any())\
                and (not data[data['item_price_level']==item_price_level]['user_occupation_id'].isnull().any()):
                    data.loc[ind,'user_occupation_id']=data[data['item_price_level']==item_price_level]['user_occupation_id'].value_counts().index[0]
                else:
                    data.loc[ind,'user_occupation_id']=data[data['shop_score_description']==shop_score_description]['user_occupation_id'].value_counts().index[0]
            else:
                data.loc[ind,'user_occupation_id']=temp['user_occupation_id'].value_counts().index[0]
            if temp['user_star_level'].isnull().any():
                data.loc[ind,'user_star_level']=data[data['shop_score_description']==shop_score_description]['user_star_level'].median()
            else:
                data.loc[ind,'user_star_level']=temp['user_star_level'].median()
    for ind in data[data['user_star_level'].isnull()].index:
        data.loc[ind,'user_star_level']=data.loc[ind-6,'user_star_level']
    for ind in data[data['user_age_level'].isnull()].index:
        data.loc[ind,'user_age_level']=data.loc[ind-6,'user_age_level']
# corr[corr['item_brand_id']>0.06]['item_brand_id'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    for ind in data[data['item_brand_id'].isnull()].index:
        shop_id=data.loc[ind,'shop_id']
        temp=data[data['shop_id']==shop_id]
        if len(temp['item_brand_id'].isnull())==len(temp):
            data.loc[ind,'item_brand_id']=data.loc[ind-1,'item_brand_id']
        else:
            data.loc[ind,'item_brand_id']=temp['item_brand_id'].value_counts().index[0]
# corr[corr['item_city_id']>0.006]['item_city_id'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    for ind in data[data['item_city_id'].isnull()].index:
        item_price_level=data.loc[ind,'item_price_level']
        temp=data[data['item_price_level']==item_price_level]
        data.loc[ind,'item_city_id']=temp['item_city_id'].value_counts().index[0]
# corr[corr['item_sales_level']>0.1]['item_sales_level'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    for ind in data[data['item_sales_level'].isnull()].index:
        item_collected_level=data.loc[ind,'item_collected_level']
        item_pv_level=data.loc[ind,'item_pv_level']
        temp=data[(data['item_collected_level']==item_collected_level)&(data['item_pv_level']==item_pv_level)]
        if temp['item_sales_level'].isnull().any():
            data.loc[ind,'item_sales_level']=data[data['item_collected_level']==item_collected_level]['item_sales_level'].median()
        else:
            data.loc[ind,'item_sales_level']=temp['item_sales_level'].median()
# corr[corr['user_gender_id']>0.01]['user_gender_id'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    count=0
    for ind in data[data['user_gender_id'].isnull()].index:
        count+=1
        user_age_level=data.loc[ind,'user_age_level']
        temp=data[data['user_age_level']==user_age_level]
        if (len(temp)==0) and (len(data[data['user_age_level']==user_age_level]['user_gender_id'])!=0):
            data.loc[ind,'user_gender_id']=data[data['user_age_level']==user_age_level]['user_gender_id'].value_counts().index[0]
        elif temp['user_gender_id'].isnull().any():
            data.loc[ind,'user_gender_id']=0
        else:
            data.loc[ind,'user_gender_id']=temp['user_gender_id'].value_counts().index[0]
        if count%1000==0:
            print(count,end=' ')
train.to_csv(r'D:\Data\IJCAI\no_fixedday\train_misspre.csv',index=False)
test.to_csv(r'D:\Data\IJCAI\no_fixedday\test_misspre.csv',index=False)
print('Done!')

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 Done!


## 处理时间过长将之前处理好的广告商品和预测属性和类目数量和比率复制过来

In [5]:
import pandas as pd
import numpy as np
tra=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\train_misspre.csv')
tes=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\test_misspre.csv')
train=pd.read_csv(r'D:\Data\IJCAI\no_fixed\train_mispret.csv')
test=pd.read_csv(r'D:\Data\IJCAI\no_fixed\test_mispret.csv')
print(len(tra),len(train),len(set(tra['instance_id'].values)),len(set(train['instance_id'].values)))
tra=tra.sort_values(by=['instance_id'])
train=train.sort_values(by=['instance_id'])
tra.index=np.arange(len(tra))
train.index=np.arange(len(train))
for col in ['com_pre_cate', 'com_pre_pro', 'com_pre_cateprob', 'com_pre_proprob']:
    tra[col]=train[col]
    tes[col]=test[col]
tra.to_csv(r'D:\Data\IJCAI\no_fixedday\train_mispret.csv',index=False)
tes.to_csv(r'D:\Data\IJCAI\no_fixedday\test_mispret.csv',index=False)

## 广告商品和预测属性和类目数量和比率

In [7]:
# import time
# import math
# import pandas as pd
# import numpy as np
# train=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\train_misspre.csv')
# test=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\test_misspre.csv')
# s=time.time()
# combine=[train,test]
# for data in combine:
#     data['com_pre_cate']=0
#     data['com_pre_pro']=0
#     data['com_pre_cateprob']=0
#     data['com_pre_proprob']=0
#     data['num_true_property']=0
#     data['num_pred_category']=0
#     data['num_pred_property']=0
#     for ind in data.index:
#         Category=data.loc[ind,'item_category_list'].split(';')
#         Property=data.loc[ind,'item_property_list'].split(';')
#         liscate=[]
#         lispro=[]
#         for i in data.loc[ind,'predict_category_property'].split(';'):
#             liscate.append(i.split(':')[0])
#             lispro.extend(i.split(':')[1].split(','))
#         while '1000' in Category:
#             Category.remove('1000')
#         while '-1' in lispro:
#             lispro.remove('-1')
#         comcate=len(set(Category)&set(liscate))
#         compro=len(set(Property)&set(lispro))
#         data.loc[ind,['com_pre_cate','com_pre_pro','com_pre_cateprob','com_pre_proprob','num_true_property',\
#                       'num_pred_category','num_pred_property']]=[comcate,compro,comcate/math.sqrt(len(Category)*len(liscate)+1),\
#                                                                  compro/math.sqrt(len(Property)*len(lispro)+1),len(Property),\
#                                                                  len(liscate),len(lispro)]
#         if (ind %10000==0) and (ind!=0):
#             print(ind,end=' ')
# e=time.time()
# print(e-s)
# train.to_csv(r'D:\Data\IJCAI\no_fixedday\train_mispret.csv',index=False)
# test.to_csv(r'D:\Data\IJCAI\no_fixedday\test_mispret.csv',index=False)

10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 10000 50647.33700275421


## 将'item_category_list'列分成三列，并去掉列值相同的'item_category_list_dup'列

In [25]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\train_mispret.csv')
test=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\test_mispret.csv')
j=0
for data in combine:
    for ind in data.index:
        if len(data.loc[ind,'item_category_list'].split(';'))==2:
            data.loc[ind,'item_category_list']=data.loc[ind,'item_category_list']+';1000'
        if (ind%10000==0) and (ind!=0):
            print(ind,end=' ')
    #进行分列赋值操作
    data['item_category_list_dup'], data['item_category_list_0'],data['item_category_list_1'] = zip(*data['item_category_list'].map(lambda x: x.split(';')))
    data.drop(['item_category_list_dup'],axis=1,inplace=True)
train.to_csv(r'D:\Data\IJCAI\no_fixedday\train_parse.csv',index=False)
test.to_csv(r'D:\Data\IJCAI\no_fixedday\test_parse.csv',index=False)

10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 10000 

### 将'predict_category_property'列分成属性和类目分别对待
***
- 对类目下的属性不是-1的并且train和test共有的类目填充增列
- 属性需要满足在共有类目下、train和test共有、并且倒排表中属性所在的行数不超过10万行不低于1900行

In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\train_parse.csv')
test=pd.read_csv(r'D:\Data\IJCAI\no_fixedday\test_parse.csv')
h_=train['predict_category_property'].apply(lambda x: x.split(';'))
h=test['predict_category_property'].apply(lambda x: x.split(';'))
listrain=[]
listrainpro=[]
#对train和test分别提取类目，属性不为-1
for i in h_:
    for j in i:
        if j.split(':')[1]!='-1':
            listrain.append(j.split(':')[0])
listest=[]
listestpro=[]
for i in h:
    for j in i:
        if j.split(':')[1]!='-1':
            listest.append(j.split(':')[0])
com=set(listrain)&set(listest)             #共有的类目
print('len(com)',len(com))
#对train和test分别提取属性，属性必须是共有类目下的属性
for i in h_:
    for j in i:
        if (j.split(':')[1]!='-1') and (j.split(':')[0] in com):
            listrainpro.extend(j.split(':')[1].split(','))

for i in h:
    for j in i:
        if (j.split(':')[1]!='-1') and (j.split(':')[0] in com):
            listestpro.extend(j.split(':')[1].split(','))
######################################################################
comm=set(listrainpro)&set(listestpro)     #共有的属性
print('len(comm)',len(comm))
#对属性建立倒排表，对每个属性，找出他出现的行值index
dic={}                                                            
for ind in train.index:
    for j in train.loc[ind,'predict_category_property'].split(';'):
        if (j.split(':')[1]!='-1') and (j.split(':')[0] in com):
            for k in j.split(':')[1].split(','):
                if k in comm:
                    if int(k) not in dic.keys():
                        dic[int(k)]=[]
                        dic[int(k)].append(ind)
                    else:
                        dic[int(k)].append(ind)
dicte={}
for ind in test.index:
    for j in test.loc[ind,'predict_category_property'].split(';'):
        if (j.split(':')[1]!='-1') and (j.split(':')[0] in com):
            for k in j.split(':')[1].split(','):
                if k in comm:
                    if int(k) not in dicte.keys():
                        dicte[int(k)]=[]
                        dicte[int(k)].append(ind)
                    else:
                        dicte[int(k)].append(ind)
#对倒排表进行排序（可选的），对每个属性出现的行数进行计数，并设置阈值，找到合适的填充列数
dick=sorted(dic.items(),key=lambda x:x[0])
dictek=sorted(dicte.items(),key=lambda x:x[0])
num_dic={}
for i,k in dick:
    num_dic[i]=len(k)
new=[]
for key,value in num_dic.items():
    if (value<100000) and (value>1900):
        new.append(key)
print('需要填充的属性列数',len(new))

In [ ]:
#对类目进行分列并填充
combine=[train,test]
######################################################################
for data in combine:
    for i in com:
        data['predict_category_%s'%str(i)]=0
combine=[train,test]
print('Done!')
######################################################################
for data in combine:
    for ind in data.index:
        for j in data.loc[ind,'predict_category_property'].split(';'):
            if (j.split(':')[1]!='-1') and (j.split(':')[0] in com):
                data.loc[ind,'predict_category_%s'%str(j.split(':')[0])]=1
        if (ind%10000==0) and (ind!=0):
            print(ind,end=' ')
print('cate has Done!')
######################################################################

Done!
10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 

In [ ]:
#对属性进行分列并填充
print('Waiting for train...')
count=0
for i in new:
    train['predict_property_%s'%str(i)]=0
for key,value in dic:
    if key not in new:
        continue
    else:
        train.loc[value,'predict_property_%s'% str(key)]=1
        count+=1
    if count%10==0:
        print(count,end=' ')
print('train has been done!')
train.to_csv(r'D:\Data\IJCAI\no_fixedday\train_parseo.csv',index=False)

count=0
for i in new:
    test['predict_property_%s'%str(i)]=0
for key,value in dicte:
    if key not in new:
        continue
    else:
        test.loc[value,'predict_property_%s'% str(key)]=1
        count+=1
    if count%10==0:
        print(count,end=' ')
print('test has been done!')
test.to_csv(r'D:\Data\IJCAI\no_fixedday\test_parseo.csv',index=False)